In [ ]:
import os
from pathlib import Path

import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.ndimage import rotate
from scipy import signal

from skimage import color, data, restoration

from utils.utils import (get_slice,
                         get_metadata,
                         mip_threeview,
                         apply_lut,
                         _idx_from_coord)
from annotation.annotations_io import load_annotations

In [ ]:
path = Path(r'C:\Users\Mahdi\OneDrive - Northeastern University\Cynthia Chai\PS9111\microfluidics\01_22_2021\animal_3\run_1\processed')

In [ ]:
def plot_2d(x, grid):
    plt.figure(figsize=(10, 10))
    plt.grid(grid)
    plt.imshow(x)
    plt.show()
    
def plot_3d(v, grid=True):
    r = v[0]
    g = v[1]
    if v.shape[0] == 3:
        b = v[2]
    else:
        b = np.zeros_like(r)
    plot_2d(np.stack([r, g, b], axis=-1), grid)
    
def plot_3d_cv2(v, grid=False):
    plot_2d(np.stack([v[:,:,2], v[:,:,1], v[:,:,0]], axis=-1), grid)

def plot_4d(v, grid=True):
    r = mip_threeview(v[0])
    g = mip_threeview(v[1])
    b = np.zeros_like(r)
    plot_2d(np.stack([r, g, b], axis=-1), grid)
    
def get_rotated(v, angle=0):
    return rotate(v[...], angle, axes=(2,3), reshape=True)
    
            
def get_aligned(v):
    output = np.zeros_like(v)
    output[1,...] = v[1, ...]
    mip_v = np.max(v, axis=1)
    
    w = np.eye(2, 3, dtype=np.float32)
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5000,  1e-10)
    
    input_mask = np.uint8(mip_v[1] > -1)
    (_, w) = cv2.findTransformECC(mip_v[1], mip_v[0], w, cv2.MOTION_EUCLIDEAN,
                                  criteria, input_mask, 5)
    for z in range(v.shape[1]):
        output[0, z, :, :] = cv2.warpAffine(v[0, z, :, :], w, (v.shape[3], v.shape[2]),
                                            flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
    return output



def gkern(kernlen=3, std=1):
    gkern1d = signal.gaussian(kernlen, std=std).reshape(kernlen, 1)
    gkern2d = np.outer(gkern1d, gkern1d)
    return gkern2d / np.sum(gkern2d)

def sharpen(frame, ct):
    kernel = np.array([[-1, -1, -1],
                       [-1, ct, -1],
                       [-1, -1, -1]])
    return signal.convolve2d(frame, kernel, 'same')
    
    
def blur(frame, kernel_size=3, kernel_sigma=1):
    kernel = gkern(kernlen=kernel_size, std=kernel_sigma)
    return signal.convolve2d(frame, kernel, 'same')

def denoise(frame, kernel_size=3, kernel_sigma=1 ,iterations=30):
    kernel = gkern(kernlen=kernel_size, std=kernel_sigma)
    temp = signal.convolve2d(frame, kernel, 'same')
    return restoration.richardson_lucy(temp, kernel, iterations)

In [ ]:
raw_vol = get_slice(path, 277)
plot_4d(raw_vol)

In [ ]:
flip_x = False
flip_y = False

if flip_x:
    raw_vol = np.flip(raw_vol, (1, 2))
if flip_y:
    raw_vol = np.flip(raw_vol, (1, 3))

plot_4d(raw_vol)

In [ ]:
rotated_vol = get_rotated(raw_vol, angle=0)
rotated_vol = np.clip(rotated_vol, raw_vol.min(), raw_vol.max())

plot_4d(rotated_vol)

In [ ]:
z_cropped_vol = rotated_vol[:, 1:-1, ...]

plot_4d(z_cropped_vol)

In [ ]:
aligned_vol = get_aligned(z_cropped_vol)
aligned_vol = np.clip(aligned_vol, raw_vol.min(), raw_vol.max())

plot_4d(aligned_vol)

In [ ]:
cropped_vol = aligned_vol[:,:,50:150,100:300]
plot_4d(cropped_vol, grid=True)

In [ ]:
threshold_vol = np.zeros_like(cropped_vol)

for c in range(cropped_vol.shape[0]):
    gamma_corrected = cropped_vol[c] ** 0.5
    temp = np.clip((gamma_corrected - np.quantile(gamma_corrected, 0.97)) / (gamma_corrected.max() - np.quantile(gamma_corrected, 0.97)) , 0.0, 1.0)
    threshold_vol[c] = temp ** 2.0

plot_4d(cropped_vol)    
plot_4d(threshold_vol)
temp = np.stack([threshold_vol[0] ** 0.45, threshold_vol[1] ** 0.45])
plot_4d(temp)

In [ ]:
scale = (10, 10, 10)

mip_z = np.dstack((np.zeros_like(np.max(threshold_vol[0], axis=(0))),
                   np.max(threshold_vol[1], axis=(0)),
                   np.max(threshold_vol[0], axis=(0))))
mip_y = np.dstack((np.zeros_like(np.max(threshold_vol[0], axis=(1))),
                   np.max(threshold_vol[1], axis=(1)),
                   np.max(threshold_vol[0], axis=(1))))
mip_x = np.transpose(np.dstack((np.zeros_like(np.max(threshold_vol[0], axis=(2))),
                                np.max(threshold_vol[1], axis=(2)),
                                np.max(threshold_vol[0], axis=(2)))), (1, 0, 2))

resized_z = cv2.resize(mip_z, (scale[2] * mip_z.shape[1], scale[1] * mip_z.shape[0]))
resized_y = cv2.resize(mip_y, (scale[2] * mip_y.shape[1], scale[0] * mip_y.shape[0]))
resized_x = cv2.resize(mip_x, (scale[0] * mip_x.shape[1], scale[1] * mip_x.shape[0]))

scaled_y = cv2.resize(mip_y, (mip_y.shape[1], 1 * mip_y.shape[0]))
scaled_x = cv2.resize(mip_x, (1 * mip_x.shape[1], mip_x.shape[0]))


plot_3d_cv2(resized_z)
plot_3d_cv2(resized_y)
plot_3d_cv2(resized_x)

In [ ]:
resized_mip = np.ones((resized_z.shape[0] + resized_y.shape[0], resized_z.shape[1] + resized_x.shape[1], 3))
resized_mip[:resized_z.shape[0], :resized_z.shape[1]] = resized_z ** 0.45
resized_mip[resized_z.shape[0]:, :resized_z.shape[1]] = resized_y ** 0.45
resized_mip[:resized_z.shape[0], resized_z.shape[1]:] = resized_x ** 0.45
plot_3d_cv2(resized_mip)

mip = np.ones((mip_z.shape[0] + scaled_y.shape[0], mip_z.shape[1] + scaled_y.shape[0], 3))
mip[:mip_z.shape[0], :mip_z.shape[1]] = mip_z ** 0.45
mip[mip_z.shape[0]:, :mip_z.shape[1]] = scaled_y ** 0.45
mip[:mip_z.shape[0], mip_z.shape[1]:] = scaled_x ** 0.45
plot_3d_cv2(mip)

In [ ]:
resized_uint8 = apply_lut(resized_mip, 0.0, 1.0, np.uint8)
uint8 = apply_lut(mip, 0.0, 1.0, np.uint8)

file_name1 = os.path.join(path, "PS9111 animal_3 resized.png")
file_name2 = os.path.join(path, "PS9111 animal_3.png")

array_name1 = os.path.join(path, "PS9111 animal_3 resized.npy")
array_name2 = os.path.join(path, "PS9111 animal_3.npy")

cv2.imwrite(file_name1, resized_uint8)
cv2.imwrite(file_name2, uint8)

np.save(array_name1, resized_uint8)
np.save(array_name2, uint8)

In [ ]:
y, x, _ = resized_z.shape
z, x, _ = resized_y.shape
sectioned_image = resized_uint8.copy()

sectioned_image[y-20:y, :, :] = 255
sectioned_image[:, x-20:x, :] = 255

plot_3d_cv2(sectioned_image)

In [ ]:
scalebar_image = sectioned_image.copy()


offset_x = 100
offset_y = 100
bar_h = 20
bar_w = 250
font_scale=1.5
thickness=5
scalebar_image[offset_y:offset_y + bar_h, offset_x:offset_x + bar_w, :] = 255
txt1 = '10 um'
(w1, h1), bl1 = cv2.getTextSize(text=txt1, fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=font_scale, thickness=thickness)
scalebar_image = cv2.putText(scalebar_image, txt1, (offset_x, offset_y + bar_h + h1 + 20), cv2.FONT_HERSHEY_SIMPLEX, 
                             font_scale, (255, 255, 255), thickness, cv2.LINE_AA)

plot_3d_cv2(scalebar_image)

In [ ]:
coordinate_image = scalebar_image.copy()
bar_h = 5
coordinate_image = cv2.arrowedLine(coordinate_image, (x-60, y-60), (x-110, y-60), 
                (255, 255, 255), bar_h, tipLength = 0.2)
coordinate_image = cv2.putText(coordinate_image, 'X', (x-140, y-50), cv2.FONT_HERSHEY_SIMPLEX, 
                             font_scale * 0.6, (255, 255, 255), thickness, cv2.LINE_AA)

coordinate_image = cv2.arrowedLine(coordinate_image, (x-60, y-60), (x-60, y-110), 
                (255, 255, 255), bar_h, tipLength = 0.2)
coordinate_image = cv2.putText(coordinate_image, 'Y', (x-70, y-130), cv2.FONT_HERSHEY_SIMPLEX, 
                             font_scale * 0.6, (255, 255, 255), thickness, cv2.LINE_AA)

coordinate_image = cv2.arrowedLine(coordinate_image, (x-60, y+40), (x-110, y+40), 
                (255, 255, 255), bar_h, tipLength = 0.2)
coordinate_image = cv2.putText(coordinate_image, 'X', (x-140, y+50), cv2.FONT_HERSHEY_SIMPLEX, 
                             font_scale  * 0.6, (255, 255, 255), thickness, cv2.LINE_AA)

coordinate_image = cv2.arrowedLine(coordinate_image, (x-60, y+40), (x-60, y+90), 
                (255, 255, 255), bar_h, tipLength = 0.2)
coordinate_image = cv2.putText(coordinate_image, 'Z', (x-70, y+130), cv2.FONT_HERSHEY_SIMPLEX, 
                             font_scale  * 0.6, (255, 255, 255), thickness, cv2.LINE_AA)

coordinate_image = cv2.arrowedLine(coordinate_image, (x+40, y-60), (x+90, y-60), 
                (255, 255, 255), bar_h, tipLength = 0.2)
coordinate_image = cv2.putText(coordinate_image, 'Z', (x+100, y-50), cv2.FONT_HERSHEY_SIMPLEX, 
                             font_scale  * 0.6, (255, 255, 255), thickness, cv2.LINE_AA)

coordinate_image = cv2.arrowedLine(coordinate_image, (x+40, y-60), (x+40, y-110), 
                (255, 255, 255), bar_h, tipLength = 0.2)
coordinate_image = cv2.putText(coordinate_image, 'Y', (x+30, y-130), cv2.FONT_HERSHEY_SIMPLEX, 
                             font_scale  * 0.6, (255, 255, 255), thickness, cv2.LINE_AA)

plot_3d_cv2(coordinate_image)

In [ ]:
zyxs = [[135,630,1020], [130,650,1210], [150,1430,1250], [155,1320,1020]]
annotated_image = coordinate_image.copy()

for i ,zyx in enumerate(zyxs):
        annotated_image = cv2.rectangle(annotated_image, (zyx[2]-55,zyx[1]-55), (zyx[2]+55,zyx[1]+55), (0,0,255), 5)
        annotated_image = cv2.rectangle(annotated_image, (zyx[2]-55,y+zyx[0]-25), (zyx[2]+55,y+zyx[0]+25), (0,0,255), 5)
        annotated_image = cv2.rectangle(annotated_image, (x+zyx[0]-25,zyx[1]-55), (x+zyx[0]+25,zyx[1]+55), (0,0,255), 5)
plot_3d_cv2(annotated_image)